### WLLS_I_estimator.ipynb
- Siep Dokter
- Emil Jousimaa
- Oleksandr Sosovskyy
- Mario Gabriele Carofano

> This file contains the implementation of 2 alternative WLLS estimators, named OS-WLLS_I (One Step) and TS-WLLS_I (Two Step), as requested in the 4th task, used for estimate the Target Coordinates starting from the RSS information coming from the anchors.

> In addition, at the end of file, there are also plots showing the actual position of the target and the anchors, and the estimated position of the target obtained from the execution of the 2 estimators.

In [ ]:
# IMPORTS
import import_ipynb
import constants
import auxfunc
import pandas as pd
import numpy as np
import pprint
import math

In [ ]:
def calculate_z_vector(G, Phi, h):
	'''
	
	'''

	pass

In [ ]:
def calculate_G_matrix():
	'''
	
	'''

	pass

In [ ]:
def calculate_Phi_matrix(K, AI, CI):
	'''
	
	'''

	pass

In [ ]:
def calculate_K_matrix(Lambda_WLLS):
	'''
	
	'''

	pass

In [ ]:
def calculate_h_vector(Lambda_WLLS):
	'''
	
	'''

	pass

In [ ]:
def calculate_AI_matrix(anchor_coords):
	'''
	
	'''

	pass

In [ ]:
def calculate_bI_vector(anchor_coords, estimated_distances):
	'''
	
	'''

	pass

In [ ]:
def calculate_CI_matrix(estimated_distances):
	'''
	
	'''

	pass

In [ ]:
def calculate_Lambda_WLLS_matrix(AI, bI, CI):
	'''
	
	'''

	pass

In [ ]:
def calculate_OS_WLLS_I_output(Lambda_WLLS):
    '''
    
    '''

    pass

In [ ]:
def calculate_TS_WLLS_I_output(Lambda_WLLS, z):
    '''
    
    '''

    pass

In [ ]:
def apply_WLLS_I_estimator(directory_path):
    '''
    
    '''

    pass

In [ ]:
data = {}

data["Scenario A"] = {}
data["Scenario B"] = {}
data["Scenario C"] = {}

data["Scenario A"]["RPI-WiFi"] = apply_WLLS_I_estimator(constants.SCENARIO_A_PATH + "RPI/RSS_WiFi_Dataset/")
data["Scenario A"]["RPI-BLT"] = apply_WLLS_I_estimator(constants.SCENARIO_A_PATH + "RPI/RSS_BLT_Dataset/")
data["Scenario A"]["RPI-Hybrid"] = apply_WLLS_I_estimator(constants.SCENARIO_A_PATH)

data["Scenario B"]["RPI-WiFi"] = apply_WLLS_I_estimator(constants.SCENARIO_B_PATH + "RPI/RSS_WiFi_Dataset/")
data["Scenario B"]["RPI-BLT"] = apply_WLLS_I_estimator(constants.SCENARIO_B_PATH + "RPI/RSS_BLT_Dataset/")
data["Scenario B"]["RPI-Hybrid"] = apply_WLLS_I_estimator(constants.SCENARIO_B_PATH)

data["Scenario C"]["RPI-WiFi"] = apply_WLLS_I_estimator(constants.SCENARIO_C_PATH + "RPI/RSS_WiFi_Dataset/")
data["Scenario C"]["RPI-BLT"] = apply_WLLS_I_estimator(constants.SCENARIO_C_PATH + "RPI/RSS_BLT_Dataset/")
data["Scenario C"]["RPI-Hybrid"] = apply_WLLS_I_estimator(constants.SCENARIO_C_PATH)

# np.set_printoptions(suppress=True)
# pprint.pprint(data)
# for configuration in data["Scenario A"]["RPI-WiFi"]["Estimated"].values():
#     pprint.pprint(configuration["Target Coordinates"])

In [ ]:
print("Scenario A:")
print("𝜭W:", auxfunc.calculate_rmse(data["Scenario A"]["RPI-WiFi"]))
print("𝜭B:", auxfunc.calculate_rmse(data["Scenario A"]["RPI-BLT"]))
print("𝜭H:", auxfunc.calculate_rmse(data["Scenario A"]["RPI-Hybrid"]), "\n")

print("Scenario B:")
print("𝜭W:", auxfunc.calculate_rmse(data["Scenario B"]["RPI-WiFi"]))
print("𝜭B:", auxfunc.calculate_rmse(data["Scenario B"]["RPI-BLT"]))
print("𝜭H:", auxfunc.calculate_rmse(data["Scenario B"]["RPI-Hybrid"]), "\n")

print("Scenario C:")
print("𝜭W:", auxfunc.calculate_rmse(data["Scenario C"]["RPI-WiFi"]))
print("𝜭B:", auxfunc.calculate_rmse(data["Scenario C"]["RPI-BLT"]))
print("𝜭H:", auxfunc.calculate_rmse(data["Scenario C"]["RPI-Hybrid"]))

In [ ]:
auxfunc.plot_data("WLLS-II", "Scenario A", data["Scenario A"])
auxfunc.plot_data("WLLS-II", "Scenario B", data["Scenario B"])
auxfunc.plot_data("WLLS-II", "Scenario C", data["Scenario C"])